In [1]:
using StatisticalRethinking, Distributed, JLD
using Mamba

# Data
line = Dict{Symbol, Any}()

howell1 = CSV.read(joinpath(dirname(Base.pathof(StatisticalRethinking)), "..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults

In [2]:
df2 = filter(row -> row[:age] >= 18, df);
line[:x] = convert(Array{Float64,1}, df2[:weight]);
line[:y] = convert(Array{Float64,1}, df2[:height]);
line[:xmat] = convert(Array{Float64,2}, [ones(length(line[:x])) line[:x]])

# Model Specification
model = Model(
  y = Stochastic(1,
    (xmat, beta, s2) -> MvNormal(xmat * beta, sqrt(s2)),
    false
  ),
  beta = Stochastic(1, () -> MvNormal([178, 0], [sqrt(10000), sqrt(100)])),
  s2 = Stochastic(() -> Uniform(0, 50))
)


# Initial Values
inits = [
  Dict{Symbol, Any}(
    :y => line[:y],
    :beta => [rand(Normal(178, 100)), rand(Normal(0, 10))],
    :s2 => rand(Uniform(0, 50))
  )
  for i in 1:3
]

# Tuning Parameters
scale1 = [0.5, 0.25]
summary1 = identity
eps1 = 0.5

scale2 = 0.5
summary2 = x -> [mean(x); sqrt(var(x))]
eps2 = 0.1

# Define sampling scheme

scheme = [
  Mamba.NUTS([:beta]),
  Mamba.Slice([:s2], 10)
]

setsamplers!(model, scheme)

# MCMC Simulation

sim = mcmc(model, line, inits, 10000, burnin=1000, chains=3)

MCMC Simulation of 10000 Iterations x 3 Chains...

Chain 1:   0% [0:11:53 of 0:11:54 remaining]
Chain 1:  10% [0:00:52 of 0:00:57 remaining]
Chain 1:  20% [0:00:50 of 0:01:02 remaining]
Chain 1:  30% [0:00:44 of 0:01:03 remaining]
Chain 1:  40% [0:00:38 of 0:01:03 remaining]
Chain 1:  50% [0:00:31 of 0:01:03 remaining]
Chain 1:  60% [0:00:25 of 0:01:03 remaining]
Chain 1:  70% [0:00:19 of 0:01:03 remaining]
Chain 1:  80% [0:00:13 of 0:01:03 remaining]
Chain 1:  90% [0:00:06 of 0:01:03 remaining]
Chain 1: 100% [0:00:00 of 0:01:02 remaining]

Chain 2:   0% [0:00:02 of 0:00:02 remaining]
Chain 2:  10% [0:00:33 of 0:00:37 remaining]
Chain 2:  20% [0:00:37 of 0:00:46 remaining]
Chain 2:  30% [0:00:35 of 0:00:50 remaining]
Chain 2:  40% [0:00:31 of 0:00:51 remaining]
Chain 2:  50% [0:00:26 of 0:00:52 remaining]
Chain 2:  60% [0:00:21 of 0:00:53 remaining]
Chain 2:  70% [0:00:16 of 0:00:54 remaining]
Chain 2:  80% [0:00:11 of 0:00:55 remaining]
Chain 2:  90% [0:00:06 of 0:00:55 remaining]
Cha

Object of type "Mamba.ModelChains"

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

[27.0607 112.793 0.921667; 26.7217 113.153 0.931622; … ; 27.2826 111.812 0.950566; 25.5009 111.562 0.959515]

[27.7904 113.67 0.905221; 26.7399 113.427 0.911362; … ; 24.4854 114.588 0.894131; 25.8016 117.542 0.829485]

[28.6413 111.875 0.948985; 28.7924 111.644 0.956488; … ; 25.5808 114.409 0.898168; 25.7584 113.926 0.901789]

Show draws summary

In [3]:
describe(sim)

Iterations = 1001:10000
Thinning interval = 1
Chains = 1,2,3
Samples per chain = 9000

Empirical Posterior Estimates:
            Mean         SD        Naive SE       MCSE        ESS   
     s2  26.1694456 1.998284374 0.01216117142 0.0199750535 9000.0000
beta[1] 113.9394486 1.912330881 0.01163807512 0.0354531353 2909.4894
beta[2]   0.9037664 0.042053414 0.00025592893 0.0007721172 2966.4434

Quantiles:
            2.5%        25.0%       50.0%       75.0%       97.5%   
     s2  22.56081082  24.7686452  26.0545835  27.4503277  30.3928628
beta[1] 110.28063651 112.6166219 113.9132797 115.1894841 117.8218397
beta[2]   0.81751585   0.8760952   0.9044646   0.9330347   0.9842561



*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*